<hr style="border-top: 3px double #8c8b8b;"/>
<br/>
<center><font size=7 color="green">データサイエンスII   第9回</font><br/>
2021.6.8
</center>
<hr style="border-top: 3px double #8c8b8b;"/>


+ 学籍番号：
+ 氏名：
+ rainbow id:

# 準備

## ライブラリ読み込み

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy as sy

from IPython.core.interactiveshell import InteractiveShell

#InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

## 関数 

### count_groups: データセットのグループの大きさ

In [ ]:
def count_groups(df,groups,as_index=True):
    temp=df.groupby(groups,as_index=as_index).count()
    xlen=len(groups)
    if as_index:
        temp1=temp.iloc[:,[0]]
        temp1.columns=["個数"]
    else:
        temp1=temp.iloc[:,range(xlen+1)]
        temp1.columns=groups+["個数"]
    return temp1

### 一変数最小値探索

In [ ]:
def minimize_simple(f,bounds=[-10,10],ndiv=100):
    import numpy as np
    a,b=bounds
    L=np.linspace(a,b,ndiv)
    fmin=f(a)
    xmin=a
    for t in L[1:]:
        if f(t)<fmin:
            fmin=f(t)
            xmin=t
    return xmin,fmin
        
    

### 探索関数 searchn

In [ ]:
def inbounds(x,bounds):
    """ bounds[i][0]<= x[i] <= bounds[i][1] の時 True
    """
    N=len(x)
    if len(bounds)!=N:
        return False
    else:
        for i in range(N):
                min,max=bounds[i]
                if min>x[i] or max < x[i]:
                    return False
        return True

def searchnbd2(f,x,nbds,epsilon,bounds):
    """  ランダム方向探索
    ランダムな回転gを選び、min { f(k g@v) | u ∈ nbds, -100<k<200, kg@v ∈ bounds} を与える点をy とし、
    f(y) < f(x ) の時は y を返し、f(y)=f(x)の時は [] を返す.
    """
    from scipy.stats import special_ortho_group
    if len(x)>1:
        g=special_ortho_group(dim=len(x)).rvs()
        gnbds=[g@u for u in nbds]
    else:
        gnbds=nbds
    fmin=f(x)  
    fmin0=fmin
    x1=x
    for gu in gnbds:
        for t in np.arange(-10,10):
            if debug:
#                val=f(x+t*epsilon*gu)                
                print("t={},epsilon={},gu={},x+t*epsilon*gu={},fmin0={},val={}".format(t,epsilon,gu,x+t*epsilon*gu,fmin0,f(x+t*epsilon*gu)))
            if inbounds(x+t*epsilon*gu,bounds):
                val=f(x+t*epsilon*gu)
                if val<fmin0:
                    fmin0=val
                    x1=x+t*epsilon*gu
            
    if fmin0<fmin:
        return x1
    else:
        return []
def searchnbd1(f,x,nbds,epsilon,bounds):
    """  ランダム方向探索
    ランダムな回転gを選び、min { f(k g@v) | u ∈ nbds, -100<k<200, kg@v ∈ bounds} を与える点をy とし、
    f(y) < f(x ) の時は y を返し、f(y)=f(x)の時は [] を返す.
    """
    gnbds=nbds
    fmin=f(x)  
    fmin0=fmin
    x1=x
    for gu in gnbds:
        for t in np.arange(-10,10):
            if debug:
#                val=f(x+t*epsilon*gu)                
                print("t={},epsilon={},gu={},x+t*epsilon*gu={},fmin0={},val={}".format(t,epsilon,gu,x+t*epsilon*gu,fmin0,f(x+t*epsilon*gu)))
            if inbounds(x+t*epsilon*gu,bounds):
                val=f(x+t*epsilon*gu)
                if val<fmin0:
                    fmin0=val
                    x1=x+t*epsilon*gu
            
    if fmin0<fmin:
        return x1
    else:
        return []

def showlist(x,format):
        str=""
        count=0
        for u in x:
            if count>0:
                str+=" ,"
            str+=format
        return str.format()

def unit_vector(k,i):
    import numpy as np
    if i<0 or k<=i:
        return unit_vector(k,0)
    else:
        u=np.zeros(k)
        u[i]=1
    return u

def searchn(f,n,nbds=[],init=[],fmin=-10000000.0,bounds=[],epsilon=1,epsilon_min=0.00000001,K=20,\
            showsteps=True,Ndir=1,xformat="{:7.5f}",xformat2="{:11.10f}",jumpProb=0.1):
    """ searchn n変数関数の最小値を、ランダム方向に探索する
    f: 目的関数, n: 目的関数の変数の数, nbds=[] :探索方向のリスト, init=[]: 初期探索点
    epsilon=1: 初期の探索幅, epsilon_min=0.00000001(10^-8)
    bounds=[]: 探索範囲
    K=20: 探索回数上限、showteps=True 途中経過表示、Ndir=1: ランダム方向の探索数上限
    """
    from random import randint
    if debug:
        print("f([0])=",f([0]))
    if len(nbds)==0:
        nbds=[]
        for i in range(n):
            nbds.append(unit_vector(n,i))
    if len(init)==0:
        init=np.array( [np.random.random() for i in range(n)])
    x=init
    history=[x]
    history_epsilon=[epsilon]
    count=0
    xmin=f(init)
    ntrial=0
    ntotal=0
    if showsteps:
        print("epsilon={},epsilon_min={},count={},K={}".format(epsilon,epsilon_min,count,K))
    if debug:
        print("epsilon={},epsilon_min={},count={},K={},x={},f(x)={},fmin={}".format(epsilon,epsilon_min,count,K,x,f(x),fmin))
    while epsilon>epsilon_min and count<K and f(x)>fmin:        
        import random
        if random.random()<jumpProb :
            epsilon1=1
        else:
            epsilon1=epsilon
        temp=searchnbd2(f,x,nbds,epsilon1,bounds)
        if showsteps:
            print("2：temp={},count={},epsilon={}".format(temp,count,epsilon))
        if len(temp)==0:
            if ntrial>=Ndir:
                epsilon*=0.1
                ntrial=0
            else:
                ntrial+=1
        else:
            ntrial=0
            x=temp
            epsilon=epsilon1
            history.append(x)
            history_epsilon.append(epsilon)
            count+=1
            if showsteps:
                print("\n減少",count)
                temp=0
                for u in x:
                    if temp>0:
                        print(",  ",end="",sep="")
                    print("x[{:1d}]=".format(temp),end="",sep="")
                    temp+=1
                    print(xformat.format(u),end="",sep="")
                print("\nf = "+xformat2.format(f(x)),"\n epsilon ={:10.11f}".format(epsilon))
    return x,f(x),epsilon,count,history_epsilon,history 

### 混同行列 confusion_matrix 

+ u によりOKとされたとき1,そうでないとき0とするコラムを作成

In [ ]:
def getvalues(df,x):
    return np.unique(df[x].values)

def confusion_matrix(df):
    idvalues=getvalues(df,"id")
    temp=[[ len(df[np.all([df.id==xid,df.hantei==h],axis=0)])  for h in [1,0]] for xid in idvalues]
    table=pd.DataFrame(temp,columns=["Y","N"])
    table["all"]=table.Y+table.N
    tableT=table.T
    tableT["total"]=tableT.apply(sum,axis=1)
    return tableT.T

### 菖データ定義関数 mk_ayame

In [ ]:
def mk_ayame():
    import plotly.express as px
    df = px.data.iris()
    df.columns=["花弁長","花弁幅","萼長","萼幅","種名","id"]
    return df

#クラスiのみ"1"他は"0"とする
def mk_ayame1(i):
    import plotly.express as px
    df = px.data.iris()
    df.columns=["花弁長","花弁幅","萼長","萼幅","種名","id0"]
    df["id"]=df.apply(lambda x: "1" if x.id0==i else "0",axis=1)
    return df

# ２平面での分類

## 方針

+  $g_c>0$ and $h_d>0$で id=1 を判定
  失点は 
     id(x)=1 のときは、$H(-g(x))+H(-h(x))$
     $id(x)\neq1$ のときは、 g(x)>0 かつ h(x)>0 のときエラー


   $loss(x,g,h)= \sum_{id(x)=1,g(x)<0 \lor h(x)<0}H(-g(x))+H(-h(x))+\sum_{id(x)\neq 1,g(x)>0, h(x)>0} (g(x)+h(x))$  

  `loss(df,g,h)=np.sum(df.apply(lambda x: loss(x,g,h)))`

In [ ]:
def ff(coeff,X):
    X=np.array(X)
    coeff=np.array(coeff)
    n=len(X)
    return X@coeff[:n]+coeff[-1] 
    

H=lambda x:x if x>0 else 0
    # X=[x,y],coeff=[a,b,c]
def loss21(X,xid,c):
    n=len(c)//2
    c1=c[:n]
    c2=c[n:]
    v1=ff(c1,X)
    v2=ff(c2,X)
    if xid=="1":
         return H(-v1)+H(-v2)
    elif xid!="1" and v1>0 and v2>0:
         return v1+v2
    else:
        return 0

def loss2(dfD,coeff,C):
    n=len(coeff)//2-1 
    lossA=np.sum(dfD.apply(lambda d : loss21(d[:n], d.id, coeff),axis=1))
    return lossA +C*((np.sum(np.array(coeff[:n])**2)-1)**2+(np.sum(np.array(coeff[n:])**2)-1)**2)

## 平面データの例

### データ定義

In [ ]:
dfD91=pd.DataFrame(np.array([2*np.random.random()-1 for i in range(2000)]).reshape(1000,2),columns=["x","y"])

dfD91["id"]=dfD91.apply(lambda x: "1" if (2*x.x+3*x.y+1>0.2) and (x.x-x.y>-1) else "0" ,axis=1)

#dfD80=dfD80[dfD80.id!="0"]
dfD91.to_csv("dfD91.csv")

In [ ]:
fig=px.scatter(dfD91,x="x",y="y",color="id")
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_layout(width=500)

In [ ]:
print("dfD91 の x+y+1,x-y-1 に関する失点は", loss2(dfD91,[1,1,1,-1,-1,1,1],10))

### 探索

In [ ]:
debug=False
bounds=[[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]]
result=searchn(lambda u: loss2(dfD91,u,30),6,fmin=0,bounds=bounds,showsteps=True,epsilon=1,K=20,jumpProb=0.1)

In [ ]:
coeff=np.array(result[0]).reshape(2,3)
coeff

In [ ]:
dfD91["hantei"]=dfD91.apply(lambda x: 1 if ff(coeff[0],x[:2])>0 and ff(coeff[1],x[:2]) else 0, axis=1)

In [ ]:
confusion_matrix(dfD91)

### 図示

In [ ]:
fig=px.scatter(dfD91,x="x",y="y",color="id")
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_layout(width=500)
t=np.linspace(-1,1)
counter=0
coeff_list=np.array(result[0]).reshape(2,3)
for coeff in coeff_list:
    a,b,c=coeff
    fig.add_scatter(x=t,y=-(a*t+c)/b,name=counter)
    counter+=1
fig.show()
    

### 探索経過

In [ ]:
t=np.linspace(-1,1)
counter=0
for coeff in result[-1]:
    fig=px.scatter(dfD91,x="x",y="y",color="id")
#     fig.add_scatter(x=dfD91.x,y=dfD91.y,mode="markers",opacity=0.6)
#     fig.add_scatter(x=dfD91.x,y=dfD91.y,mode="markers",opacity=0.3)
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_layout(width=500)
    coeff=np.array(coeff).reshape(2,3)
    for cff in coeff:
        a,b,c=cff
        fig.add_scatter(x=t,y=-(a*t+c)/b,name=counter)
        counter+=1
    fig.show()
    

## 菖の第２類の分類への適用

In [ ]:
dfA=mk_ayame1(2)
count_groups(dfA,["id","id0"])

### searchnでの探索

In [ ]:
debug=False
bounds=5*np.array([[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]])
result=searchn(lambda u: loss2(dfA,u,10),10,fmin=0,bounds=bounds,showsteps=True,epsilon=1,K=20,jumpProb=0.1)

In [ ]:
coeff=np.array(result[0]).reshape(2,5)
coeff

In [ ]:
dfA["hantei"]=dfA.apply(lambda x: 1 if ff(coeff[0],x[:4])>0  and ff(coeff[1],x[:4])>0 else 0 ,  axis=1)
confusion_matrix(dfA)

In [ ]:
count_groups(dfA,["id","hantei"])

### dual_anealing での探索

In [ ]:
from scipy.optimize import dual_annealing
bounds=10*np.array([[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]])
result=dual_annealing(lambda u: loss2(dfA,u,10),bounds=bounds)
result

In [ ]:
coeff=result.x.reshape(2,5)
coeff

In [ ]:
dfA["hantei"]=dfA.apply(lambda x: 1 if ff(coeff[0],x[:4])>0  and ff(coeff[1],x[:4])>0 else 0 ,  axis=1)
confusion_matrix(dfA)

In [ ]:
count_groups(dfA,["id","hantei"])

# エントロピー

## エントロピーの基本概念


+ あるデータがとりうる値が$N$個あるとき、そのデータ値を確定する情報は、$N$個の「曖昧さ」をなくしたことになる。したがって、その情報の大きさは$N$ということができる。
$N$個の値を取り得るデータ$x,y$の値を確定する情報は、$N^2$個の曖昧さをなくしたことになり、情報の大きさは$N^2$ということができる。
$N$について単調増大な関数$f$を使って$f(N)$で情報量は量っても大小関係は変わらない。通常$\log N$を情報量とする。このとき、$x,y$のデータの情報量は$x$の情報量と$y$の情報量の和となる、という便利さがある。
+ $x$のデータ値がわかることで得られる情報量は、それが得られる前に$x$がもっていた「不定性」「曖昧さ」の度合いを表す。この不定度をエントロピーと呼ぶ。
すなわち可能性が$N$通りある状況について$\log_2 N$ をそのエントロピーと呼ぶ。



In [ ]:
entropy_temp= lambda N:np.log2(N)

In [ ]:
t=range(1,100)
temp=pd.DataFrame(np.array([t,entropy_temp(t)]).T,columns=["可能性数","エントロピー"])
temp["可能性数"]=temp["可能性数"].astype(int)
temp.set_index("可能性数")
temp[:20].set_index("可能性数")

In [ ]:
fig=px.scatter(temp,x="可能性数",y="エントロピー")
fig.add_scatter(x=temp["可能性数"],y=temp["エントロピー"])

## 基本関数の定義

In [ ]:
def entropy0(H):
    import numpy as np
    from numpy import log2
    H=np.array(H).flatten()
    return -(np.sum([p*log2(p) for p in H if p>0]))

def entropy(H):
    import numpy as np
    from numpy import array,sum
    H=array(H).flatten()
    pH=H/sum(H)
    return entropy0(pH)

In [ ]:
entropy([1,10])

In [ ]:
def entropyMarginal1(H):
    import numpy as np
    from numpy import array,sum

    H=array(H)
    H1=[sum(p) for p in H] 
    return entropy(H1)

def entropyMarginal2(H):
    import numpy as np
    H=np.array(H)
    return entropyMarginal1(H.T)


def showtable(H):
    import pandas as pd
    df=pd.DataFrame(H)
    df["計"]=df.apply(sum,axis=1)
    dfT=df.T
    dfT["計"]=dfT.apply(sum,axis=1)
    return dfT.T

def CondProb1to2(H):
    import pandas as pd
    df=pd.DataFrame(H)
    df["total"]=df.apply(sum,axis=1)
    df=df.apply(lambda x: x/x.total,axis=1).drop("total",axis=1)
    return df
    
def CondProb2to1(H):
    import numpy as np
    return CondProb1to2(np.array(H).T).T


def muinfo(H):
    import numpy as np
    H=np.array(H)
    H1=[np.sum(p) for p in H] 
    H2=[np.sum(p) for p in H.T]
    return entropy(H1)+entropy(H2)-entropy(H)

def condEntropy1to2_kobetsu(H,y):
    H=np.array(H)
    return entropy(H[y])

def condEntropy2to1_kobetsu(H,y):
    return condEntropy1to2_kobetsu(H.T,y)

def condEntropy1to2(H):
    H1=[np.sum(p) for p in H]
    return entropy(H)-entropy(H1)
   
def condEntropy2to1(H):
    H=np.array(H).T
    return condEntropy1to2(H)

def showentropy(H):
    import numpy as np
    H=np.array(H)
    display(showtable(H))
    HX=[np.sum(p) for p in H] 
    HY=[np.sum(p) for p in H.T]
    print("H(V,X)={:3.2f}".format(entropy(H)))
    print("H(V)={:3.2f}".format(entropyMarginal1(H)))
    print("H(X)={:3.2f}".format(entropyMarginal2(H)))
    print("I(V,X)={:3.2f}".format(muinfo(H)))
    print("H(V | X)={:3.2f}".format(condEntropy2to1(H)))
    print("H(V | X)/H(V)={:3.2f}".format(condEntropy2to1(H)/entropyMarginal1(H)))

## 同時分布のエントロピー諸量例

### 例

In [ ]:
H=np.array([[1,0],[0,1]])
showentropy(H)


In [ ]:
display(CondProb1to2(H),CondProb2to1(H))

### 例

In [ ]:
H=np.array([[1,1],[0,1]])
showentropy(H)


In [ ]:
display(CondProb1to2(H),CondProb2to1(H))

### 例

In [ ]:
H=np.array([[90,1],[0,100]])
showentropy(H)
display(CondProb1to2(H),CondProb2to1(H))

### 例

In [ ]:
H=np.array([[1,0,1],[0,1,0]])
showentropy(H)


In [ ]:
display(CondProb1to2(H),CondProb2to1(H))

In [ ]:
H=np.array([[1,0,1],[0,1,0]])
showentropy(H.T)


### 例

In [ ]:
H=np.array([[4,1],[1,4],[0,5]])
showentropy(H)

In [ ]:
display(CondProb1to2(H),CondProb2to1(H))

## (p,1-p)のエントロピーの観察


### グラフ

In [ ]:
t=np.linspace(0.01,0.99)
df=pd.DataFrame(np.array([t,-(t*np.log2(t)+(1-t)*np.log2(1-t))]).T,columns=["p","entropy"])
fig=px.line(df,x="p",y="entropy")
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_layout(width=500)

### [1,k]のエントロピー

In [ ]:
px.line(x=range(1,20),y=[entropy([1,k]) for k in range(1,20)])

+ エントロピーが0.3 のとき、すでに[1,18]の偏りであることに注意

In [ ]:
pd.DataFrame([[t,entropy([t,1-t])] for t in np.arange(0.00,0.5,0.01)],columns=["p","(p,1-p)のエントロピー"])

### 逆エントロピー

In [ ]:
def inventropy(H):
    for i in np.arange(1000,1,-1):
        if entropy([1,i])>H:
            return i

In [ ]:
t=np.arange(0.1,1.0,0.01)
s=[inventropy(x) for x in t]
df=pd.DataFrame(np.array([t,s]).T,columns=["H([1,N])","N"])
px.line(df,x="H([1,N])",y="N")

## データフレームの属性のエントロピー

+ データ集合$df$の属性$x$の値が$0,1,\cdots,k-1$とし、$x=i$となるデータの個数を$n_i$とする.
+ このとき、分布$(n_i)_{0\leq i<k}$のエントロピーを、属性$x$のエントロピーと呼ぶ.

In [ ]:
def df_entropy(df,x):
    u=df.groupby(x).count().values[:,0]
    return entropy(u)

def df_hist(df,x,values):
    return [len(df[df[x]==u]) for u in values]
    


### 菖データの例　

In [ ]:
dfA=mk_ayame()
dfA.columns=[0,1,2,3,"種名","id"]

+ 菖データでは種属性のエントロピーは$\log_23$である。

In [ ]:
df_entropy(dfA,"id")

In [ ]:
df_hist(dfA,"id",[1,2,3])

# 菖データで、属性を二分したときの条件付きエントロピーを求める

## 第i属性$x_i$を$x_i-t$の正負で二分した情報についての id の条件付きエントロピー

### 関数 mk_table

+ $i$番目の属性値が$t$より大きいとき$Y$,そうでないとき$N$とするDataFrame を作る

In [ ]:
def mk_table(dfA,i,t):
    dfA["hantei"]=dfA.apply( lambda x: 1 if x[i]>t  else 0,axis=1)
    return confusion_matrix(dfA)

In [ ]:
mk_table(dfA,0,5)

In [ ]:
mk_table(dfA,0,6)

### 関数 cond_entropy

+ df.i > t の真偽に関する id のエントロピー

In [ ]:
def cond_entropy(df,i,t):
    H=mk_table(df,i,t).values
    H=H[:-1,:-1]
    return condEntropy2to1(H)

# 属性j で条件づけた属性iのエントロピー
def cond_entropy_2to1(df,i,j):
    H=df[[i,j]]
    return condEntropy2to1(H)

In [ ]:
import plotly.express as px
t=np.linspace(0,10)
# fig=px.line(x=t,y=[determinacy(dfA,0,k) for k in t])
fig=px.scatter()
fig.add_scatter(x=t,y=[cond_entropy(dfA,0,k) for k in t],name="id エントピー | 属性0")
fig.add_scatter(x=t,y=[cond_entropy(dfA,1,k) for k in t],name="id エントピー | 属性1")
fig.add_scatter(x=t,y=[cond_entropy(dfA,2,k) for k in t],name="id エントピー | 属性2")
fig.add_scatter(x=t,y=[cond_entropy(dfA,3,k) for k in t],name="id エントピー | 属性3")
fig.show()


## 最も良い判定を与える属性と属性値を求める

In [ ]:
for i in range(4):
    u=minimize_simple(lambda t: cond_entropy(dfA,i,t),bounds=[0,max(dfA[i])])
    print("属性{}: 最小残留エントロピー={:3.2f},分岐値={:5.4}".format(i, u[1],u[0]))

In [ ]:
display(mk_table(dfA,2,1.952),mk_table(dfA,3,0.6061))

i=2,3のいずれかが情報量が大きい

## <font color="red">属性２で判別する</font>

In [ ]:
xid=2
shikii=1.952

dfA["hantei"]=dfA.apply(lambda x: 1 if x[xid] > shikii else 0, axis=1)

confusion_matrix(dfA)

## 決定木の記録

In [ ]:
tree=pd.DataFrame([[0,-1,0,[50,50,50],df_entropy(dfA,"id"),xid,shikii]],columns=["node","boss","branch","bunpu","entropy","var","shikii"])
tree

In [ ]:
boss=0
for i in range(2):
    dftemp=dfA[dfA["hantei"]==1-i]
    N=len(tree)
    tree.loc[N]=[N,boss,1-i,df_hist(dftemp,"id",[1,2,3]),df_entropy(dftemp,"id"),0,0]
tree

## <font color="red">entropy が正の場合について同じことを行う</font>

### dfA1 の定義

In [ ]:
dfA1=dfA[dfA["hantei"]==1]
df_hist(dfA1,"id",[1,2,3])

In [ ]:
dftemp=dfA1.copy()
import plotly.express as px
t=np.linspace(0,10)
# fig=px.line(x=t,y=[determinacy(dfA,0,k) for k in t])
fig=px.scatter()
fig.add_scatter(x=t,y=[cond_entropy(dftemp,0,k) for k in t],name="0成分残留エントロピー")
fig.add_scatter(x=t,y=[cond_entropy(dftemp,1,k) for k in t],name="1成分残留エントロピー")
fig.add_scatter(x=t,y=[cond_entropy(dftemp,2,k) for k in t],name="2成分残留エントロピー")
fig.add_scatter(x=t,y=[cond_entropy(dftemp,3,k) for k in t],name="3成分残留エントロピー")
fig.show()


In [ ]:
for i in range(4):
    u=minimize_simple(lambda t: cond_entropy(dftemp,i,t),bounds=[0,dftemp[i].max()])
    print("属性{}: 最小残留エントロピー={:3.2f},分岐値={:5.4f}".format(i, u[1],u[0]))

### 属性3 について t=1.7 で判別する

In [ ]:
xid=3
t=1.7172

In [ ]:
tree.loc[1,"var"]=xid
tree.loc[1,"shikii"]=t
tree

In [ ]:
dfA["hantei1"]=dfA["hantei"]
dfA["hantei"]=dfA.apply(lambda x: 1 if x[xid]>t else 0,axis=1)
dfA["hantei2"]=dfA["hantei"]

In [ ]:
confusion_matrix(dfA)

In [ ]:
boss=1
for i in [1,0]:
    N=len(tree)
    dftemp=dfA[np.all([dfA["hantei1"]==1,dfA["hantei2"]==i],axis=0)]
    tree.loc[N]=[N,boss,i,df_hist(dftemp,"id",[1,2,3]),df_entropy(dftemp,"id"),0,0]

In [ ]:
tree

## 以上をまとめる

In [ ]:
dfA["all"]=dfA.apply(lambda x: "1" if x["hantei1"]==0 else "01" if x["hantei2"]==1 else "00", axis=1)

In [ ]:
H=dfA.groupby(["id","all"])[0].count().unstack().fillna(0)
display(H)
condEntropy2to1(H.values)

In [ ]:
pd.DataFrame([[u,len(dfA[dfA["all"]==u]),df_entropy(dfA[dfA["all"]==u],"id")] for u in getvalues(dfA,"all")],columns=["判定","個数","残留エントロピー"])

# 演習 wine のデータ

## データ集合 dfwine の定義

In [ ]:
from sklearn.datasets import load_wine
winedata=load_wine()

dfwine=pd.DataFrame(winedata.data)
dfwine.columns=winedata.feature_names
dfwine["id"]=winedata.target
dfwine.columns=list(range(len(dfwine.columns)-1))+["id"]
dfwine.head()

In [ ]:
len(dfwine)

## 基本的な量

### ワインのクラス

In [ ]:
getvalues(dfwine,"id")

### クラスのエントロピー

In [ ]:
df_entropy(dfwine,"id")

In [ ]:
df_hist(dfwine,"id",[0,1,2])

## 分離属性の探索

In [ ]:
import plotly.express as px
t=np.linspace(0,10)
fig=px.scatter()
for i in range(13):
    fig.add_scatter(x=t,y=[cond_entropy(dfwine,i,k) for k in t],name="id エントピー | 属性{}".format(i))
fig.show()


In [ ]:
for i in range(13):
    u=minimize_simple(lambda t: cond_entropy(dfwine,i,t),bounds=[min(dfwine[i]),max(dfwine[i])])
    print("属性{}: 最小残留エントロピー={:3.2f},分岐値={:5.4f}".format(i, u[1],u[0]))

## 課題

+ 属性6 で閾値 1.5848 で　$\S4.3$と同様のことを行う
+ 二分したそれぞれのデータ集合について$\S4.5$と同様のことを行う